<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [29]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Darwin':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

El dataset ya se encuentra descargado


In [30]:
# Posibles bandas
os.listdir("./songs_dataset/")

['prince.txt',
 'dickinson.txt',
 'notorious-big.txt',
 'beatles.txt',
 'bob-dylan.txt',
 'bjork.txt',
 'johnny-cash.txt',
 'disney.txt',
 'janisjoplin.txt',
 'kanye.txt',
 'bob-marley.txt',
 'leonard-cohen.txt',
 'ludacris.txt',
 'adele.txt',
 'alicia-keys.txt',
 'joni-mitchell.txt',
 'amy-winehouse.txt',
 'lorde.txt',
 'rihanna.txt',
 'Kanye_West.txt',
 'nirvana.txt',
 'cake.txt',
 'bieber.txt',
 'notorious_big.txt',
 'missy-elliott.txt',
 'dolly-parton.txt',
 'jimi-hendrix.txt',
 'michael-jackson.txt',
 'al-green.txt',
 'lil-wayne.txt',
 'lady-gaga.txt',
 'lin-manuel-miranda.txt',
 'nursery_rhymes.txt',
 'dj-khaled.txt',
 'radiohead.txt',
 'patti-smith.txt',
 'blink-182.txt',
 'Lil_Wayne.txt',
 'dr-seuss.txt',
 'r-kelly.txt',
 'drake.txt',
 'britney-spears.txt',
 'bruce-springsteen.txt',
 'nicki-minaj.txt',
 'kanye-west.txt',
 'paul-simon.txt',
 'nickelback.txt',
 'eminem.txt',
 'bruno-mars.txt']

In [31]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/eminem.txt', sep='/n', header=None)
df.head()

/var/folders/k9/vb534jt106d2plphjzl86xcw0000gn/T/ipykernel_26436/3634877419.py:2: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,0
0,"Look, I was gonna go easy on you and not to hu..."
1,But I'm only going to get this one chance
2,"Something's wrong, I can feel it (Six minutes,..."
3,"Just a feeling I've got, like something's abou..."
4,"If that means, what I think it means, we're in..."


In [32]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 6812


### 1 - Preprocesamiento

In [33]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [34]:
# Demos un vistazo
sentence_tokens[:2]

[['look',
  'i',
  'was',
  'gonna',
  'go',
  'easy',
  'on',
  'you',
  'and',
  'not',
  'to',
  'hurt',
  'your',
  'feelings'],
 ['but', "i'm", 'only', 'going', 'to', 'get', 'this', 'one', 'chance']]

### 2 - Crear los vectores (word2vec)

In [35]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [36]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=4,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [37]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [38]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 6812


In [39]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 1347


### 3 - Entrenar embeddings

In [40]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 153601.265625
Loss after epoch 1: 110694.796875
Loss after epoch 2: 110327.625
Loss after epoch 3: 111146.5
Loss after epoch 4: 108832.4375
Loss after epoch 5: 108822.5625
Loss after epoch 6: 104264.3125
Loss after epoch 7: 99165.8125
Loss after epoch 8: 99017.375
Loss after epoch 9: 92187.0625
Loss after epoch 10: 85758.625
Loss after epoch 11: 85379.625
Loss after epoch 12: 87024.25
Loss after epoch 13: 84762.875
Loss after epoch 14: 83528.375
Loss after epoch 15: 79953.125
Loss after epoch 16: 84847.75
Loss after epoch 17: 81179.0
Loss after epoch 18: 82852.375
Loss after epoch 19: 82153.0


(785504, 1296760)

### 4 - Ensayar

In [41]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["god"], topn=10)

[('food', 0.79974764585495),
 ('stamps', 0.7679835557937622),
 ('damn', 0.7394956350326538),
 ('ooh', 0.7273403406143188),
 ('swear', 0.7255299091339111),
 ('yea', 0.7025995850563049),
 ('thank', 0.6947539448738098),
 ('acid', 0.6889259815216064),
 ('ah', 0.6799051761627197),
 ('doc', 0.6674500107765198)]

In [42]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["feeling"], topn=10)

[('my', -0.21762771904468536),
 ('there', -0.22507089376449585),
 ('he', -0.23031049966812134),
 ('please', -0.23288622498512268),
 ('these', -0.23990227282047272),
 ("can't", -0.24519146978855133),
 ('her', -0.24610267579555511),
 ("won't", -0.25203749537467957),
 ('here', -0.25444096326828003),
 ('will', -0.25548532605171204)]

In [43]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["palabra"])

KeyError: "Key 'palabra' not present in vocabulary"

In [ ]:
# el método `get_vector` permite obtener los vectores:
vector_god = w2v_model.wv.get_vector("god")
print(vector_god)

[ 0.38016084  0.20893666  0.00216159  0.05827052  0.3203117   0.21140929
  0.03084468  0.54755545 -0.07380334 -0.05966441  0.17423275 -0.17357948
 -0.21035829  0.17758012 -0.16173925 -0.32555273  0.09546147  0.081723
  0.12102413 -0.35852048 -0.04664207 -0.26383057  0.16393869 -0.23625582
  0.31787857 -0.19644243 -0.64424706 -0.16914718  0.10211243 -0.02551625
 -0.1957734   0.2701539   0.22284667 -0.14525034 -0.01007587  0.45852068
 -0.0575407   0.01171747 -0.18547083  0.1736652  -0.09147227  0.01652293
 -0.17871882 -0.30341303  0.20040779  0.436997    0.22134307 -0.16906476
  0.3396696   0.12286977  0.08473019 -0.29426834 -0.0435686   0.29691207
  0.21160726  0.00325255  0.1203628  -0.15581098 -0.03029534  0.01003821
  0.2047989  -0.26750422  0.20341091 -0.2004146  -0.15454479 -0.22402681
  0.3109417   0.3103943   0.14394078 -0.23691966  0.01676925 -0.15181048
 -0.03050179 -0.43056297  0.2183166  -0.15482907 -0.3322059  -0.10640176
  0.10078683  0.09747347  0.17788418  0.01679448  0.2

In [ ]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_god)

[('god', 1.0000001192092896),
 ('food', 0.809460461139679),
 ('damn', 0.7668165564537048),
 ('stamps', 0.7651999592781067),
 ('ooh', 0.728290319442749),
 ('swear', 0.7206417322158813),
 ('acid', 0.7076746821403503),
 ('thank', 0.7018301486968994),
 ('yea', 0.6960077285766602),
 ('star', 0.6919614672660828)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["god"], topn=10)

[('food', 0.809460461139679),
 ('damn', 0.7668165564537048),
 ('stamps', 0.7651999592781067),
 ('ooh', 0.728290319442749),
 ('swear', 0.7206417322158813),
 ('acid', 0.7076746821403503),
 ('thank', 0.7018301486968994),
 ('yea', 0.6960077285766602),
 ('star', 0.6919614672660828),
 ('ah', 0.6837462782859802)]

### 5 - Visualizar agrupación de vectores

In [ ]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show()

In [ ]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show()

In [ ]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/

vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.